In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [ ]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layer = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=0, stride=1)
        self.relu1 = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten = nn.Flatten()
        # The input size for the first linear layer needs to be calculated.
        # Input shape: (3, 200, 200)
        # After conv_layer: (32, 200 - 3 + 1, 200 - 3 + 1) = (32, 198, 198)
        # After max_pool: (32, 198 / 2, 198 / 2) = (32, 99, 99)
        self.linear1 = nn.Linear(32 * 99 * 99, 64)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(64, 1)
        # Removed sigmoid for BCEWithLogitsLoss

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.relu1(x)
        x = self.max_pool(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu2(x)
        x = self.output_layer(x)
        return x

model = SimpleCNN()
print(model)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
print(optimizer)

In [ ]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

dummy_input = torch.randn(1, 3, 200, 200).to(device)
summary(model, input_size=(3, 200, 200), device=str(device))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")